In [0]:
!git clone https://github.com/facebookresearch/ParlAI.git
!cd ParlAI; python setup.py develop
!pip install 'git+https://github.com/rsennrich/subword-nmt.git#egg=subword-nmt'
!pip install -U discord.py

In [0]:
import pexpect
class ChatHandler:
    def __init__(self):
        self.child = pexpect.spawn('python ParlAI/parlai/scripts/interactive.py -t blended_skill_talk -mf zoo:blender/blender_90M/model', timeout=None)
        self.child.expect('Enter Your Message:')
        self.personality = self.child.before.decode('utf-8', 'ignore').split('[context]')[1]
    def listen(self):
        response = self.child.before
        resp = response.split(b'1m')
        respfinal = resp[1].split(b'\x1b')
        return respfinal[0].decode('utf-8')
    def say(self, message):
        self.child.sendline(message)
        self.child.expect('Enter Your Message:')

In [0]:
!python ParlAI/parlai/scripts/interactive.py -t blended_skill_talk -mf zoo:blender/blender_90M/model

In [0]:
import os

import discord

TOKEN = "PUT YOUR TOKEN HERE"

client = discord.Client()

chat = ChatHandler()
print("Chat Inited")
@client.event
async def on_message(message):
    if message.content == 'chattrpersona':
        print(chat.personality)
        await message.channel.send(chat.personality)
    if message.content.startswith("chattr "):
        print("Intercepted Message")
        saying = message.content[7:]
        chat.say(saying)
        response = chat.listen()
        await message.channel.send(response)

client.run(TOKEN)

In [0]:
import pexpect
class ChatHandler:
    def __init__(self):
        self.child = pexpect.spawn('python ParlAI/parlai/scripts/interactive.py -t blended_skill_talk -mf zoo:blender/blender_90M/model', timeout=None)
        self.child.expect('Enter Your Message:')
        self.personality = self.child.before.decode('utf-8', 'ignore').split('[context]')[1]
    def listen(self):
        response = self.child.before
        resp = response.split(b'1m')
        respfinal = resp[1].split(b'\x1b')
        return respfinal[0].decode('utf-8')
    def say(self, message):
        self.child.sendline(message)
        self.child.expect('Enter Your Message:')

import discord
TOKEN = "PUT YOUR TOKEN HERE"
class Chattr(discord.Client):
    async def on_ready(self):
        self.server_chats = dict()
        for server in self.guilds:
            self.server_chats[server.name] = ChatHandler()
    async def on_message(self, message):
        print(message.content)
        print(message.author.id)
        print(self.user.id)
        if message.content == 'chattrpersona':
            await message.channel.send(chat.personality)
        if message.content == 'chattrforget':
            self.server_chats[message.guild.name].say('[DONE]')
            await message.channel.send('Chat history forgotten.')
        if message.content.startswith("chattr "):
            saying = message.content[7:]
            self.server_chats[message.guild.name].say(saying)
            response = self.server_chats[message.guild.name].listen()
            await message.channel.send(response)
client = Chattr()
client.run(TOKEN)